In [72]:
import pandas as pd
import seaborn as sns
import plotly.express as px
import datetime
import pre_process
import constants
from pyspark.sql.functions import when, lit, to_date

In [101]:
from pre_process import create_notifications_df

notification_df = create_notifications_df()
del notification_df['endTime']

# only want dates between '2017-03-01' and '2017-09-01'
filtered_notification_df = notification_df[(notification_df['startTime'] >= pd.to_datetime('2017-03-01')) & (notification_df['startTime'] <= pd.to_datetime('2017-09-01'))]
filtered_notification_df

,externalId,dataSetId,startTime,type,subtype,description,assetIds,source,id,lastUpdatedTime,createdTime
2,DWH_SAP_Notification_000400050847,7226141328114146,2017-05-14 14:34:33,Notification,M2,Oljesvetting fra se-glass - WIP,[4228372676749099],DWH_SAP_Notifications,436844235029952,2022-01-06 14:59:06.794,2020-05-13 12:02:16.238
3,DWH_SAP_Notification_000400050967,7226141328114146,2017-05-20 14:34:37,Notification,M2,Feil pÃ¥ 29TCV3014,[2487560746996654],DWH_SAP_Notifications,4181139007284667,2022-01-06 12:24:04.358,2020-05-13 12:11:20.426
4,DWH_SAP_Notification_000400051050,7226141328114146,2017-05-23 14:34:38,Notification,M2,Olje lekkasje pÃ¥ gjengene,[2009492469639655],DWH_SAP_Notifications,7112333331142709,2022-01-06 14:57:49.115,2020-05-13 12:22:38.720
5,DWH_SAP_Notification_000400051061,7226141328114146,2017-05-24 14:34:39,Notification,M2,Motor start feiler pÃ¥ mistet overtrykk.,[4228372676749099],DWH_SAP_Notifications,4810505339604182,2022-01-06 12:24:08.182,2020-05-13 12:22:38.708
6,DWH_SAP_Notification_000400051955,7226141328114146,2017-07-03 14:35:16,Notification,M2,Termografivinduer defekt,[5055494010832641],DWH_SAP_Notifications,1768095144416583,2022-01-06 12:24:09.156,2020-05-13 12:18:07.412


In [74]:
motor_subsets = pre_process.run(mode=constants.MOTOR)
de_bearing_temp = motor_subsets['de_bearing_temp']

fetching data...
creating subsets...


In [95]:
# Filter out 6 months of data
# splitting up the two different sensors and resample

# filter data where the date is greater than 2015-03-14
def filter_and_resample(df, start_date = '2017-03-01', end_date = '2017-09-01'): 
    # todo: find all unique labels in df and split up based on these
    filtered_dates = df.filter((df["timestamp"] >= lit(start_date)) & (df["timestamp"] <= lit(end_date))) 
    a = filtered_dates.filter(filtered_dates['label'] == 'DE Bearing Temp A').toPandas().set_index('timestamp').resample('60min').mean().interpolate()
    b = filtered_dates.filter(filtered_dates['label'] == 'DE Bearing Temp B').toPandas().set_index('timestamp').resample('60min').mean().interpolate()

    new_df = pd.DataFrame(index=a.index)
    new_df['DE Bearing Temp A'] = a['data']
    new_df['DE Bearing Temp B'] = b['data']
    return new_df


In [96]:
filtered_de = filter_and_resample(de_bearing_temp)
filtered_de


/Users/ane/Projects/MasterThesis/master/master-thesis/venv/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning:

Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead

/var/folders/9w/xmv0pp696b3dv7bw9rk02fb40000gn/T/ipykernel_96248/4089675886.py:8: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

/Users/ane/Projects/MasterThesis/master/master-thesis/venv/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning:

Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead

/var/folders/9w/xmv0pp696b3dv7bw9rk02fb40000gn/T/ipykernel_96248/4089675886.py:9: FutureWarning:

The default value of numeric_only in DataFr

,DE Bearing Temp A,DE Bearing Temp B
timestamp,,
2017-03-01 00:00:00,9.998631,8.998857
2017-03-01 01:00:00,9.998631,8.998857
2017-03-01 02:00:00,9.998631,8.998857
2017-03-01 03:00:00,9.998631,8.998857
2017-03-01 04:00:00,9.998631,8.998857
...,...,...
2017-08-31 19:00:00,32.999779,31.398222
2017-08-31 20:00:00,31.798772,29.997261
2017-08-31 21:00:00,30.330518,28.997486


In [102]:
fig = px.line(filtered_de, x=filtered_de.index, y=filtered_de[filtered_de.columns[0]])
fig.add_scatter(x=filtered_de.index, y=filtered_de[filtered_de.columns[1]])



for start_time in filtered_notification_df['startTime']:
    fig.add_vrect(x0=start_time -  pd.Timedelta(days=3), x1=start_time + pd.Timedelta(days=3), line_width=0, fillcolor="red", opacity=0.2)
fig.show()

23/02/28 07:09:06 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 2678114 ms exceeds timeout 120000 ms
23/02/28 07:09:06 WARN SparkContext: Killing executors is not supported by current scheduler.
23/02/28 07:09:06 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.